In [ ]:
from coinbase.rest import RESTClient
from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:
api_key = "<YOUR_API_KEY>"
api_secret = "<YOUR_API_SECRET>"

In [ ]:
client = RESTClient(api_key=api_key, api_secret=api_secret)

In [ ]:
#REST
from coinbase.rest import RESTClient
from concurrent.futures import ThreadPoolExecutor, as_completed
import json

# Initialize best bid and ask values
btc_usd_bid = None
btc_usd_ask = None
eth_btc_bid = None
eth_btc_ask = None
eth_usd_bid = None
eth_usd_ask = None

# Function to fetch market trades
def get_market_trades(product_id):
    return client.get_market_trades(product_id=product_id, limit=1)

# Product IDs for each step of the arbitrage cycle
product_ids = ["BTC-USD", "ETH-BTC", "ETH-USD"]

# Use ThreadPoolExecutor to parallelize fetching market prices
with ThreadPoolExecutor(max_workers=3) as executor:
    # Submit tasks to the executor
    futures = {executor.submit(get_market_trades, pid): pid for pid in product_ids}

    # Collect results as they are completed
    for future in as_completed(futures):
        product_id = futures[future]
        try:
            data = future.result()
            print(f"Market trades for {product_id}: {data}")

            # Check if the response data is a string or dictionary
            if isinstance(data, str):
                response_data = json.loads(data)
            elif isinstance(data, dict):
                response_data = data
            else:
                raise ValueError("Unexpected response format")

            best_bid = float(response_data['best_bid'])
            best_ask = float(response_data['best_ask'])

            # Update best bid and ask values for each product
            if product_id == "BTC-USD":
                btc_usd_bid = best_bid
                btc_usd_ask = best_ask
            elif product_id == "ETH-BTC":
                eth_btc_bid = best_bid
                eth_btc_ask = best_ask
            elif product_id == "ETH-USD":
                eth_usd_bid = best_bid
                eth_usd_ask = best_ask
        except Exception as exc:
            print(f"{product_id} generated an exception: {exc}")

# Print the best bid and ask values for each product
print("Best bid and ask for BTC-USD: ", btc_usd_bid, btc_usd_ask)
print("Best bid and ask for ETH-BTC: ", eth_btc_bid, eth_btc_ask)
print("Best bid and ask for ETH-USD: ", eth_usd_bid, eth_usd_ask)

# Arbitrage calculation
amount = 10
final_val = amount * (btc_usd_ask * eth_btc_bid * (1 / eth_usd_bid))

if final_val > amount:
    print("Arbitrage: ", final_val)
else:
    print("Insufficient data for arbitrage calculation.")
    print("Final value: ", final_val)


In [ ]:
from coinbase.rest.accounts import get_accounts
from coinbase.rest import RESTClient

# Initialize REST client
client = RESTClient(api_key=api_key, api_secret=api_secret)

# Function to extract account information
def extract_account_info():
    try:
        # Get accounts information
        accounts_info = get_accounts(client)
        print("Accounts Information:")
        print(accounts_info)
    except Exception as exc:
        print("Error fetching account information:", exc)

# Call the function to extract account information
extract_account_info()


In [ ]:
from coinbase.rest.orders import limit_order_gtc_buy, limit_order_gtc_sell
from coinbase.rest import RESTClient
from concurrent.futures import ThreadPoolExecutor, as_completed
import json

# Initialize best bid and ask values
btc_usd_bid = None
btc_usd_ask = None
eth_btc_bid = None
eth_btc_ask = None
eth_usd_bid = None
eth_usd_ask = None

client = RESTClient(api_key=api_key, api_secret=api_secret)

# Function to fetch market trades
def get_market_trades(product_id):
    return client.get_market_trades(product_id=product_id, limit=1)

# Product IDs for each step of the arbitrage cycle
product_ids = ["BTC-USD", "ETH-BTC", "ETH-USD"]

# Precision settings
price_precision = {
    "BTC-USD": 0.01,
    "ETH-BTC": 0.00001,
    "ETH-USD": 0.01
}
base_size_precision = 0.00000001

# Use ThreadPoolExecutor to parallelize fetching market prices
with ThreadPoolExecutor(max_workers=3) as executor:
    while True:
        try:
            # Submit tasks to the executor
            futures = {executor.submit(get_market_trades, pid): pid for pid in product_ids}

            # Collect results as they are completed
            for future in as_completed(futures):
                product_id = futures[future]
                try:
                    data = future.result()
                    print(f"Market trades for {product_id}: {data}")

                    # Check if the response data is a string or dictionary
                    if isinstance(data, str):
                        response_data = json.loads(data)
                    elif isinstance(data, dict):
                        response_data = data
                    else:
                        raise ValueError("Unexpected response format")

                    best_bid = float(response_data['best_bid'])
                    best_ask = float(response_data['best_ask'])

                    # Update best bid and ask values for each product
                    if product_id == "BTC-USD":
                        btc_usd_bid = best_bid
                        btc_usd_ask = best_ask
                    elif product_id == "ETH-BTC":
                        eth_btc_bid = best_bid
                        eth_btc_ask = best_ask
                    elif product_id == "ETH-USD":
                        eth_usd_bid = best_bid
                        eth_usd_ask = best_ask
                except Exception as exc:
                    print(f"{product_id} generated an exception: {exc}")

            # Print the best bid and ask values for each product
            print("Best bid and ask for BTC-USD: ", btc_usd_bid, btc_usd_ask)
            print("Best bid and ask for ETH-BTC: ", eth_btc_bid, eth_btc_ask)
            print("Best bid and ask for ETH-USD: ", eth_usd_bid, eth_usd_ask)

            # Arbitrage calculation
            initial_capital = 3
            required_ret = 1
            btc_amount = round(initial_capital / btc_usd_ask, 8)  # specify precision directly
            eth_amount = round(btc_amount / eth_btc_bid, 8)       # specify precision directly
            final_usd_amount = eth_amount * eth_usd_bid # specify precision directly
            print("Final amount: ", final_usd_amount)
            if final_usd_amount > initial_capital*required_ret:
                print("Arbitrage opportunity detected. Final amount: ", final_usd_amount)

                # Place buy, sell, and sell orders
                try:
                    client_instance = RESTClient(api_key=api_key, api_secret=api_secret)
                    buy_btc_order_response = limit_order_gtc_buy(
                        client_instance,
                        client_order_id="BUY_BTC",
                        product_id="BTC-USD",
                        base_size=str(btc_amount),
                        limit_price=str(btc_usd_ask)
                    )
                    print("Buy BTC order placed:", buy_btc_order_response)
                    time.sleep(1)

                    if buy_btc_order_response['success']:
                        sell_eth_order_response = limit_order_gtc_sell(
                            client_instance,
                            client_order_id="SELL_ETH",
                            product_id="ETH-BTC",
                            base_size=str(eth_amount),
                            limit_price=str(eth_btc_bid)
                        )
                        time.sleep(1)

                    print("Sell ETH order placed:", sell_eth_order_response)
                    if sell_eth_order_response['success']:
                        sell_usd_order_response = limit_order_gtc_sell(
                            client_instance,
                            client_order_id="SELL_USD",
                            product_id="ETH-USD",
                            base_size=str(final_usd_amount),
                            limit_price=str(eth_usd_bid)
                        )
                    print("Sell USD order placed:", sell_usd_order_response)
                except Exception as exc:
                    print("Error placing orders:", exc)
            else:
                print("No arbitrage opportunity detected.")

        except KeyboardInterrupt:
            print("Keyboard interrupt detected. Stopping...")
            break
